In [2]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

2025/09/12 06:20:41 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/12 06:20:41 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[<Experiment: artifact_location='/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/2', creation_time=1757655991553, experiment_id='2', last_update_time=1757655991553, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/1', creation_time=1757654328038, experiment_id='1', last_update_time=1757654328038, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/0', creation_time=1757654328030, experiment_id='0', last_update_time=1757654328030, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: b16d1f19aad545b9974c906ae2810bae, rmse: 7.6166
run id: d011c6d504ae462bb4666d12900ee4d1, rmse: 9.2334


Model Registry



In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [8]:
client.get_run("d011c6d504ae462bb4666d12900ee4d1")

<Run: data=<RunData: metrics={'rmse': 9.23343622815571}, params={'learning_rate': '0.09585355369315604',
 'max_depth': '30',
 'min_child_weight': '1.060597050922164',
 'objective': 'reg:linear',
 'reg_alpha': '0.018060244040060163',
 'reg_lambda': '0.011658731377413597',
 'seed': '42'}, tags={'mlflow.runName': 'masked-kit-244',
 'mlflow.source.name': '/home/codespace/.local/lib/python3.12/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'codespace'}>, info=<RunInfo: artifact_uri='/workspaces/Mlops-Zoomcamp-2025/02-experiment-tracking/mlruns/1/d011c6d504ae462bb4666d12900ee4d1/artifacts', end_time=1757654916717, experiment_id='1', lifecycle_stage='active', run_id='d011c6d504ae462bb4666d12900ee4d1', run_name='masked-kit-244', start_time=1757654715337, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[], model_inputs=[]>, outputs=<RunOutputs: model_outputs=[<LoggedModelOutput: model_id='m-1e0dc97aaca5467d910e3f0f4d249d08', step

In [9]:
run_id = "d011c6d504ae462bb4666d12900ee4d1"
model_uri = f"runs:/{run_id}/models_mlflow"  # Use the actual artifact name - models_mlflow
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-model")

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2025/09/12 06:29:52 WARNING mlflow.tracking._model_registry.fluent: Run with id d011c6d504ae462bb4666d12900ee4d1 has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-1e0dc97aaca5467d910e3f0f4d249d08 instead
Created version '1' of model 'nyc-taxi-model'.


<ModelVersion: aliases=[], creation_timestamp=1757658592311, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1757658592311, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='d011c6d504ae462bb4666d12900ee4d1', run_link=None, source='models:/m-1e0dc97aaca5467d910e3f0f4d249d08', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [11]:
model_name = "nyc-taxi"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_2174/574510856.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)
